In [1]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.dataclasses import Document


c:\Users\alira\anaconda3\envs\sorbonne\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.__version__)


2.6.0+cu124


# Document Store

In [3]:
document_store = InMemoryDocumentStore()


In [4]:
documents = [
    Document(
        content="Artificial intelligence is transforming healthcare and finance.",
        meta={"lang": "en", "source": "test_en_1"}
    ),
    Document(
        content="الذكاء الاصطناعي يغير مجالات الطب والتمويل بشكل جذري.",
        meta={"lang": "ar", "source": "test_ar_1"}
    ),
    Document(
        content="Machine learning is a subset of artificial intelligence.",
        meta={"lang": "en", "source": "test_en_2"}
    ),
    Document(
        content="تعلم الآلة هو جزء من الذكاء الاصطناعي.",
        meta={"lang": "ar", "source": "test_ar_2"}
    )
]


In [5]:
splitter = DocumentSplitter(
    split_by="sentence",
    split_length=5,
    split_overlap=1
)


In [6]:
document_embedder = SentenceTransformersDocumentEmbedder(
    model="intfloat/multilingual-e5-small",
    prefix="passage: "
)
document_embedder.warm_up()


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1092.90it/s, Materializing param=pooler.dense.weight]                              
BertModel LOAD REPORT from: intfloat/multilingual-e5-small
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
writer = DocumentWriter(document_store)


In [8]:
ingestion_pipeline = Pipeline()
ingestion_pipeline.add_component("splitter", splitter)
ingestion_pipeline.add_component("embedder", document_embedder)
ingestion_pipeline.add_component("writer", writer)

ingestion_pipeline.connect("splitter", "embedder")
ingestion_pipeline.connect("embedder", "writer")


🚅 Components
  - splitter: DocumentSplitter
  - embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - splitter.documents -> embedder.documents (list[Document])
  - embedder.documents -> writer.documents (list[Document])

In [9]:
documents = [
    Document(content="Artificial intelligence is transforming healthcare."),
    Document(content="الذكاء الاصطناعي يغير مجال الرعاية الصحية."),
    Document(content="Machine learning is a subset of AI."),
    Document(content="تعلم الآلة هو جزء من الذكاء الاصطناعي.")
]


In [10]:
ingestion_pipeline.run({
    "splitter": {
        "documents": documents
    }
})


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


{'writer': {'documents_written': 4}}

In [11]:
docs = document_store.filter_documents()
len(docs), docs[0].embedding


(4,
 [0.04796324670314789,
  0.01804913952946663,
  -0.05176614969968796,
  -0.06620220839977264,
  0.09119812399148941,
  0.019064277410507202,
  0.03561483323574066,
  0.022805247455835342,
  0.023575637489557266,
  0.015585825778543949,
  0.04074416682124138,
  0.02897908352315426,
  0.08362524956464767,
  -0.055478788912296295,
  -0.03767363727092743,
  0.03409387543797493,
  0.10263031721115112,
  -0.05481068789958954,
  -0.05244922637939453,
  -0.04326128959655762,
  -0.030443893745541573,
  -0.017666863277554512,
  -0.0614592470228672,
  0.08487307280302048,
  0.06706401705741882,
  0.06405425071716309,
  -0.02701205387711525,
  -0.004717334173619747,
  0.02177138440310955,
  -0.04302538186311722,
  -0.06790175288915634,
  -0.06638488918542862,
  0.043984729796648026,
  -0.0707896500825882,
  0.024574195966124535,
  0.06187838688492775,
  -0.09372556954622269,
  -0.09639187902212143,
  0.06050123646855354,
  -0.09267812967300415,
  -0.03745802491903305,
  -0.037600062787532806,


In [12]:
len(docs[0].embedding)

384

# 02_retrieval_quality.ipynb
# Retrieval

In [13]:
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers import InMemoryEmbeddingRetriever


In [14]:
query_embedder = SentenceTransformersTextEmbedder(
    model="intfloat/multilingual-e5-small",
    prefix="query: "
)
query_embedder.warm_up()


In [15]:
retriever = InMemoryEmbeddingRetriever(document_store)


In [16]:
query_pipeline = Pipeline()
query_pipeline.add_component("embedder", query_embedder)
query_pipeline.add_component("retriever", retriever)

query_pipeline.connect("embedder", "retriever")


🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (list[float])

In [17]:
result = query_pipeline.run({
    "embedder": {"text": "ما هو الذكاء الاصطناعي؟"},
    "retriever": {"top_k": 3}
})

for doc in result["retriever"]["documents"]:
    print(doc.content)


Batches: 100%|██████████| 1/1 [00:00<00:00, 43.51it/s]

تعلم الآلة هو جزء من الذكاء الاصطناعي.
الذكاء الاصطناعي يغير مجال الرعاية الصحية.
Machine learning is a subset of AI.


In [18]:
result = query_pipeline.run({
    "embedder": {"text": "What is machine learning?"},
    "retriever": {"top_k": 3}
})

for doc in result["retriever"]["documents"]:
    print(doc.content)


Batches: 100%|██████████| 1/1 [00:00<00:00, 166.73it/s]

Machine learning is a subset of AI.
تعلم الآلة هو جزء من الذكاء الاصطناعي.
Artificial intelligence is transforming healthcare.


In [19]:
result = query_pipeline.run({
    "embedder": {"text": "Explain الذكاء الاصطناعي"},
    "retriever": {"top_k": 4}
})

for doc in result["retriever"]["documents"]:
    print(doc.content)


Batches: 100%|██████████| 1/1 [00:00<00:00, 83.35it/s]

تعلم الآلة هو جزء من الذكاء الاصطناعي.
الذكاء الاصطناعي يغير مجال الرعاية الصحية.
Machine learning is a subset of AI.
Artificial intelligence is transforming healthcare.



# LLM

In [20]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator


In [21]:
rag_query_embedder = SentenceTransformersTextEmbedder(
    model="intfloat/multilingual-e5-small",
    prefix="query: "
)
rag_query_embedder.warm_up()


In [26]:
rag_retriever = InMemoryEmbeddingRetriever(document_store)


In [22]:
prompt_template = """
You are a helpful assistant.
Use ONLY the following documents to answer the question.
If the answer is not present, say "I don't know".

Documents:
{% for doc in documents %}
{{ doc.content }}
{% endfor %}

Question: {{ query }}
Answer:
"""

prompt_builder = PromptBuilder(template=prompt_template)


PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [27]:
llm = HuggingFaceLocalGenerator(
    model="Qwen/Qwen2.5-3B-Instruct",
    task="text-generation",
    generation_kwargs={
        "max_new_tokens": 256,
        "temperature": 0.2,
        "do_sample": False
    }
)


In [ ]:
rag_pipeline = Pipeline()

rag_pipeline.add_component("query_embedder", rag_query_embedder)
rag_pipeline.add_component("retriever", rag_retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)

rag_pipeline.connect("query_embedder", "retriever")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")


PipelineError: Component has already been added in another Pipeline. Components can't be shared between Pipelines. Create a new instance instead.

In [25]:
result = rag_pipeline.run({
    "query_embedder": {"text": "ما هو الذكاء الاصطناعي؟"},
    "prompt_builder": {"query": "ما هو الذكاء الاصطناعي؟"}
})

print(result["llm"]["replies"][0])


ValueError: Component named prompt_builder not found in the pipeline.